<a href="https://colab.research.google.com/github/chaerui7967/ko_Apartment_Deal/blob/master/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGBM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 데이터 가져오기

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/project/final.csv')

In [4]:
df.head()

,Gu_Label,Dong_Label,Year,Exclusive_area,Floor,Price per squar,per_cost,top10,name_top10,YN_r,Hospital,Station,Transaction_real_price
0,0,8,32,77.75,7,7003.020394,1922.829582,0,1.0,1,0.0,1,149500
1,0,8,33,54.98,7,7003.020394,2619.134231,0,1.0,1,0.0,1,144000
2,0,8,33,79.97,7,7003.020394,2035.138177,0,1.0,1,0.0,1,162750
3,0,8,33,79.97,8,7003.020394,2000.750281,0,1.0,1,0.0,1,160000
4,0,8,33,79.97,5,7003.020394,1875.703389,0,1.0,1,0.0,1,150000


## 데이터 상관성 확인
- 면적당 가격 변수와 공시지가 변수의 다중공선성이 예상됨

In [17]:
df.corr()

,Gu_Label,Dong_Label,Year,Exclusive_area,Floor,Price per squar,per_cost,top10,name_top10,YN_r,Hospital,Station,Transaction_real_price
Gu_Label,1.000000,0.153723,-0.063263,0.073007,0.011489,0.017240,0.012496,0.053387,-0.011383,0.033958,-0.058346,-0.062922,0.042222
Dong_Label,0.153723,1.000000,-0.009175,0.033631,-0.009071,0.034677,-0.004160,0.023015,-0.027447,-0.010063,-0.130541,-0.119431,0.020126
Year,-0.063263,-0.009175,1.000000,0.007954,-0.168207,-0.023730,-0.017571,-0.414312,0.247947,-0.044628,0.131619,0.154928,-0.022844
Exclusive_area,0.073007,0.033631,0.007954,1.000000,0.111913,0.232245,0.101595,0.147455,-0.040630,0.003551,-0.050694,-0.046014,0.621976
Floor,0.011489,-0.009071,-0.168207,0.111913,1.000000,0.061861,0.103451,0.136767,-0.020222,-0.005687,-0.036092,-0.022454,0.194157
Price per squar,0.017240,0.034677,-0.023730,0.232245,0.061861,1.000000,0.660751,0.068800,-0.108250,0.063967,-0.145129,0.019658,0.633968
per_cost,0.012496,-0.004160,-0.017571,0.101595,0.103451,0.660751,1.000000,0.148406,-0.021952,0.119634,-0.087514,0.021458,0.637118
top10,0.053387,0.023015,-0.414312,0.147455,0.136767,0.068800,0.148406,1.000000,-0.132607,0.176902,-0.128476,-0.092094,0.231179
name_top10,-0.011383,-0.027447,0.247947,-0.040630,-0.020222,-0.108250,-0.021952,-0.132607,1.000000,0.061107,0.107170,0.105403,-0.048034
YN_r,0.033958,-0.010063,-0.044628,0.003551,-0.005687,0.063967,0.119634,0.176902,0.061107,1.000000,0.022781,0.106059,0.091822


## 전체 데이터 모델링

In [49]:
X = df.drop(['Transaction_real_price'], axis = 1)
y = df['Transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [50]:
%time
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [51]:
from math import sqrt
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 434985448.1827082
RMSE : 20856.304758578597


In [52]:
lgb.feature_importances_

array([ 9192,  9233, 26483, 31072,  6824, 15209, 18702,  3176,   474,
        1899,  1011,  4210])

In [53]:
X.columns

Index(['Gu_Label', 'Dong_Label', 'Year', 'Exclusive_area', 'Floor',
       'Price per squar', 'per_cost', 'top10', 'name_top10', 'YN_r',
       'Hospital', 'Station'],
      dtype='object')

## 변수 선택
- feature_importances_ 기준 2000 이하값 삭제
- 병원 유무, 지역이름 유무, 상위빈도 이름 삭제

In [54]:
df_rj = df.drop(['Hospital','name_top10', 'YN_r'], axis=1)

In [55]:
%time
X = df_rj.drop(['Transaction_real_price'], axis = 1)
y = df_rj['Transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.54 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [56]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 437075216.16076285
RMSE : 20906.343921421623


In [57]:
lgb.feature_importances_

array([ 9616,  9842, 26416, 31052,  6588, 15754, 19263,  3479,  4773])

In [58]:
X.columns

Index(['Gu_Label', 'Dong_Label', 'Year', 'Exclusive_area', 'Floor',
       'Price per squar', 'per_cost', 'top10', 'Station'],
      dtype='object')

## 다중공성선 문제인가

## 공시지가로 모델링
- 면적당 가격 제거

In [18]:
df_1 = df.drop('per_cost', axis=1)

In [19]:
X = df_1.drop(['Transaction_real_price'], axis = 1)
y = df_1['Transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [20]:
%time
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [22]:
df_1.columns

Index(['Gu_Label', 'Dong_Label', 'Year', 'Exclusive_area', 'Floor',
       'Price per squar', 'top10', 'name_top10', 'YN_r', 'Hospital', 'Station',
       'Transaction_real_price'],
      dtype='object')

In [21]:
from math import sqrt
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 538066129.5644171
RMSE : 23196.252489667746


In [27]:
lgb.feature_importances_

array([ 9151, 15931, 30460, 26755,  6522, 20161,  4342,  1357,  2471,
        2138,  7360])

# 면적당 가격
- 공시지가 제거

In [33]:
df_2 = df.drop('Price per squar', axis=1)

In [35]:
X = df_2.drop(['Transaction_real_price'], axis = 1)
y = df_2['Transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [36]:
%time
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [37]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 474760606.30988544
RMSE : 21789.001957636458


In [38]:
lgb.feature_importances_

array([22479,  9455, 24649, 30704,  5337, 25779,  2716,   369,  1788,
        1213,  4484])

## 시공사기준 + 병원 유무 
- 공시지가 제거, 높은 빈도수 이름 제거

In [44]:
df_h = df.drop(['Price per squar','name_top10'], axis=1)

In [45]:
df_h.head(1)

,Gu_Label,Dong_Label,Year,Exclusive_area,Floor,per_cost,top10,YN_r,Hospital,Station,Transaction_real_price
0,0,8,32,77.75,7,1922.829582,0,1,0.0,1,149500


In [46]:
X = df_h.drop(['Transaction_real_price'], axis = 1)
y = df_h['Transaction_real_price']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [47]:
%time
lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [48]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 475248605.7791465
RMSE : 21800.197379362107


# 공시지가, 시공사기준 top10, hospital 제거 으로 예측

In [59]:
df_3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/project/seoul_4.csv')

In [60]:
df_3.head(1)

,gu_l,dong_l,s_c,used_y,square,per_cost_man,floor,YN_r,YN_top10,cost
0,0,8,1,32,77.75,1922.829582,7,1,1,149500


In [61]:
X = df_3.drop(['cost'], axis = 1)
y = df_3['cost']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size = 0.2,
                                                   random_state = 99)

In [62]:
%time
from lightgbm import LGBMRegressor

lgb = LGBMRegressor(linear_tree = True,
                   boosting_type = 'gbdt',
                   objective = 'regression',
                   n_estimators = 9000,
                   learning_rate = 0.001,
                   max_depth = 4,
                   n_jobs = -1)

lgb.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.001, linear_tree=True,
              max_depth=4, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=9000, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [63]:
MSE = mean_squared_error(y_test, lgb.predict(X_test))

print(f'MSE : {MSE}')
print(f'RMSE : {sqrt(MSE)}')

MSE : 15002672.830508133
RMSE : 3873.3283917721374
